In [ ]:
"""
What to submit
You need to supply two circuits:

qc_init: Prepares the qubits (of which there are at least two) in a desired initial state;
qc_syn: Measures a subset of the qubits.
The artificial errors to be inserted are x and z gates on two particular qubits. You need to 
pick the two qubits to be used for this (supplied as the list error_qubits).

There are 16 possible sets of errors to be inserted (including the trivial case of no errors).
The measurement result of qc_syn should output a unique bit string for each. The grader will 
return the error message 'Please make sure the circuit is created to the initial layout.' if this is not satisfied.


The grader will compile the complete circuit for the backend ibmq_tokyo (a retired device)
. To show that your solution is tailor made for the device, this transpilation should not
change the number of cx gates. If it does, you will get the error message 'Please make sure 
the circuit is created to the initial layout.'

To guide the transpilation, you'll need to tell the transpiler which qubits on the device should 
be used as which qubits in your circuit. This is done with an initial_layout list.

You may start with the example given below, which can become a valid answer with a few tweaks.
"""
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile

import qiskit.tools.jupyter
from qiskit.test.mock import FakeTokyo
code = QuantumRegister(5,'code')
syn = QuantumRegister(4,'syn')
out = ClassicalRegister(4,'output')
qc_syn = QuantumCircuit(code,syn,out)


# Left ZZZ
qc_syn.cx(code[0],syn[1])
qc_syn.cx(code[2],syn[1])
qc_syn.cx(code[3],syn[1])


# Right ZZZ
qc_syn.swap(syn[0],code[1])
qc_syn.cx(syn[0],syn[2])
qc_syn.swap(syn[0],code[1])
qc_syn.cx(code[2],syn[2])
qc_syn.cx(code[4],syn[2])


# Top XXX
qc_syn.h(syn[0])
qc_syn.cx(syn[0],code[0])
qc_syn.cx(syn[0],code[1])
qc_syn.cx(syn[0],code[2])
qc_syn.h(syn[0])


# Bottom XXX
qc_syn.h(syn[3])
qc_syn.cx(syn[3],code[2])
qc_syn.cx(syn[3],code[3])
qc_syn.cx(syn[3],code[4])
qc_syn.h(syn[3])



# Measure the auxilliary qubits
qc_syn.measure(syn,out)
qc_syn.draw('mpl')


qc_init = QuantumCircuit(code,syn,out)

qc_init.h(syn[0])
qc_init.cx(syn[0],code[0])
qc_init.cx(syn[0],code[1])
qc_init.cx(syn[0],code[2])
qc_init.cx(code[2],syn[0])

qc_init.h(syn[3])
qc_init.cx(syn[3],code[2])
qc_init.cx(syn[3],code[3])
qc_init.cx(syn[3],code[4])
qc_init.cx(code[4],syn[3])


qc_init.draw('mpl')


qc = qc_init.compose(qc_syn)
display(qc.draw('mpl'))

job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()


error_qubits = [0,4]

def insert(errors,error_qubits,code,syn,out):

    qc_insert = QuantumCircuit(code,syn,out)

    if 'x0' in errors:
        qc_insert.x(error_qubits[0])
    if 'x1' in errors:
        qc_insert.x(error_qubits[1])
    if 'z0' in errors:
        qc_insert.z(error_qubits[0])
    if 'z1' in errors:
        qc_insert.z(error_qubits[1])
        
    return qc_insert



for error in ['x0','x1','z0','z1']:
    
    qc = qc_init.compose(insert([error],error_qubits,code,syn,out)).compose(qc_syn)
    job = Aer.get_backend('qasm_simulator').run(qc)
    
    print('\nFor error '+error+':')
    counts = job.result().get_counts()
    for output in counts:
        print('Output was',output,'for',counts[output],'shots.')
        
        
# Please use the backend given here
backend = FakeTokyo()
backend

qc = qc_init.compose(qc_syn)
qc = transpile(qc, basis_gates=['u','cx'])
qc.num_nonlocal_gates()

qc1 = transpile(qc,backend,basis_gates=['u','cx'], optimization_level=3)
qc1.num_nonlocal_gates()

initial_layout = [0,2,6,10,12,1,5,7,11]

qc2 = transpile(qc,backend,initial_layout=initial_layout, basis_gates=['u','cx'], optimization_level=3)
qc2.num_nonlocal_gates()
